In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, ConfusionMatrixDisplay

In [2]:
data=pd.read_csv("spambase.csv", header=None)

# Transformando em Numpy
val=data.values
att=data.columns
X=val[:,0:57]
y=val[:,57]

# Mostrando o formato de X e y
print(X.shape)
print(y.shape)

(4601, 57)
(4601,)


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

parameters=[{'kernel':["linear", "poly", "rbf", "sigmoid"], 
             'gamma': ["scale", "auto"], 
             'probability':[True,False]}]

# Separando uma parte dos dados para validação (usaremos no GridSearchCV)
X,X_val,y,y_val=train_test_split(X,y,test_size=0.2,random_state=42, stratify=y)

clf = SVC()

# Execução do GridSearch
gs=GridSearchCV(clf, parameters, scoring='accuracy', cv=5, n_jobs=-1)

gs.fit(X_val, y_val)

# Mostrando a tabela de resultados do GridSearch (opcional)
import pandas as pd
df=gs.cv_results_
df = pd.DataFrame(gs.cv_results_)
print("Melhores parâmetros encontrados: ", gs.best_params_)

# Recuperando os melhores resultados
clf=gs.best_estimator_
df

In [ ]:
#Métricas de avaliação

# Usando validação cruzada com T=5 folds
result=model_selection.cross_val_score(clf, X, y, cv=5)
print("Acurácia média: %.3f" % result.mean())
print("Desvio padrão: %.3f" % result.std())

# Calculando a predição para cada exempolo de teste
y_pred=model_selection.cross_val_predict(clf, X, y, cv=5)

# Calculando a predição para cada exemplo de teste
#y_pred_proba=model_selection.cross_val_predict(clf, X, y, cv=5, method='predict_proba')

# Calcular precisão
precision=precision_score(y, y_pred, average='macro')

# Calcular revocação
recall=recall_score(y, y_pred, average='macro')

# Calcular revocação
f1=f1_score(y, y_pred, average='macro')

print("Precision: %.3f" % precision)
print("Recall: %.3f" % recall)
print("f1: %.3f" % f1)

matrix=ConfusionMatrixDisplay(y, y_pred)
cm = confusion_matrix(y, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['não_spam', 'spam'])
disp.plot()